# index_points function
The index-points function is a central function within the overall model. At several stages within a single pass, the respective sampled points have to be selected and re-indexed for the next layer or function to process them. This notebook aims at specifically looking at this function with its respective inputs, as for the multi-scale-grouping (MSG) version of this model, currently we are receiving the following error: <br>
<br>
*RuntimeError: CUDA error: device-side assert triggered* <br>
<br>
This usually means, that there is something going sideways with the indices, I have found out. Therefore we will now adjust the function and test it in all its glory to finally get rid of this motherf****** error.

## Imports and Data

In [ ]:
# IMPORTS
import torch
import numpy as np

# DATA
#here I will have to insert some random tensors in the respective shape for all the layers etc, so we can test it properly

: 

## Previous definition of index_points
In order to evaluate the situation properly, I have inserted the commented and traditional verison of the function definition below. For clarity it has been renamed **index_points_old**.

In [ ]:
# Traditional definition of inddex_points
def index_points_old(points, idx):
    """
    Indexing points according to new group index.
    
    Input:
        points: input points data, [Batch/ Blocks, Num Points, Num Features]
        idx: sample index data, [Batch/ Blocks, New Num Points]
    Return:
        new_points: indexed points data, [Batches/ Blocks, New Num Points, Num Features]
    """
    device = points.device
    B = points.shape[0] # B is number of batches/ blocks
    # Defining views/ shapes for the re-indexing
    view_shape = list(idx.shape)
    view_shape[1:] = [1] * (len(view_shape) - 1)
    repeat_shape = list(idx.shape)
    repeat_shape[0] = 1
    # Batch/ Block indexing for input points according to previously defined views/ shapes
    batch_indices = torch.arange(B, dtype=torch.long).to(device).view(view_shape).repeat(repeat_shape)
    # Picking corresponding points from input points
    new_points = points[batch_indices, idx, :]
    
    # Return reindexed points according to batch/ block index and New Num Points
    return new_points

## Testable Hypotheses


Now, since the error seems to arise spontaneously at almost random iterations (or better said, to me random iterations), here you can find some ideas for what has been going sideways:<br>
- batch/block number/index does not match
- indices and points are on different devices 
- more will follow...

## Solutions

After taking a look at the testable hypotheses, and also trying to evaluate those accordingly, in the following you will find the proposed solutions and their results.

### 1. Writing own re-indexing function

As the logic of it should be rather straightforward, my first initial thought is to re-write the entire function according to my own understanding of its functionality, then subsequent debugging might also be easier. A first draft of a version can be found below:

In [ ]:
def index_points(points, idx):
    """
    Indexing points according to new group index.
    
    Input:
        points: input points data, [Batch/ Blocks, Num Points, Num Features]
        idx: sample index data, [Batch/ Blocks, New Num Points]
    Return:
        new_points: indexed points data, [Batches/ Blocks, New Num Points, Num Features]
    """
    
    device = points.device
    B = points.shape[0] # B is number of batches/ blocks
    
    # Check for index shape
    if idx.shape[0] < B:
        print(f"Batch number lower for indeces. Should be {B}, but is {idx.shape[0]}")
    elif idx.shape[0] > B:
        print(f"Batch number higher for indeces. Should be {B}, but is {idx.shape[0]}")
        
    # Defining shapes in respective dimension to be indexed
    batch_shape = sample_shape = list(idx.shape)
    batch_shape[1:] = sample_shape[0] = 1
    
    # Creating batch indices by indexing according to the shapes
    batch_indices = torch.arange(B, dtyp=torch.long).to(device).view(batch_shape).repeat(sample_shape)
    
    # Selecting corresponding point from input points
    new_points = points[batch_indices, idx, :]
    
    # Return reindexed points according to batch/ block index
    return new_points